# This is the start of the actual research notebook

### current goals:

- build a functional predictor using lightgbm
- find areas for improvement

In [241]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import lightgbm as lgb

In [242]:
# #helper
# def _rolling_slope(series: pd.Series, window: int) -> pd.Series:
#     x = np.arange(window)
#     x_mean = x.mean()
#     denom = ((x - x_mean)**2).sum()
#     def _slope_window(y):
#         y_mean = y.mean()
#         num = ((x- x_mean)*(y-y_mean)).sum()
#         return num / denom if denom != 0 else np.nan
#     return series.rolling(window=window).apply(_slope_window, raw=False)

In [243]:
# def add_rolling_features(df, windows):
#     num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

#     #remove Y1 Y2
#     for tgt in ("Y1","Y2"):
#         if tgt in num_cols:
#             num_cols.remove(tgt)
    
#     #add mean, slope, and vol features per feature
#     new_cols = []
#     for col in num_cols:
#         s = df[col]
#         s_prev = s.shift(1)

#         nonpos_share = float((s_prev<=0).sum()) / max(len(s_prev),1)
#         use_pct = nonpos_share<0.05
#         for W in windows:
#             mean_col = f"roll_mean__{col}__W{W}"
#             df[mean_col] = s_prev.rolling(window=W).mean()
#             new_cols.append(mean_col)

#             slope_col = f"roll_slope__{col}__W{W}"
#             df[slope_col] = _rolling_slope(s_prev, W)
#             new_cols.append(slope_col)

#             vol_input = s_prev.pct_change() if use_pct else s_prev.diff()
#             vol_col = f"roll_vol__{col}__W{W}"
#             df[vol_col] = vol_input.rolling(window=W).std()
#             new_cols.append(vol_col)
#     return df, new_cols

### Load and prepare data


In [244]:
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')


In [245]:
lags = [1,5,10]

feature_cols = [col for col in train_data.columns if col not in ['time', 'Y1', 'Y2']]
window_size = 5
for col in feature_cols:
    train_data[f"{col}_rollmean_{window_size}"] = train_data[col].rolling(window=window_size).mean()
for col in feature_cols:
    values = train_data[col].to_numpy()
    slopes = [np.nan] * (window_size-1)
    x = np.arange(window_size)
    for i in range(window_size -1,len(values)):
        y = values[i-window_size+1:i+1]
        slope = np.polyfit(x,y,1)[0]
        slopes.append(slope)
    train_data[f"{col}_rollslope_{window_size}"] = pd.Series(slopes, index=train_data.index)
for col in feature_cols:
    train_data[f"{col}_rollvol_{window_size}"] = (
        train_data[col]
        .rolling(window=window_size, min_periods=window_size)
        .std(ddof=0)
    )
for col in feature_cols:
    for lag in lags:
        train_data[f"{col}_lag{lag}"] = train_data[col].shift(lag)
for col in feature_cols:
    train_data[f"{col}^2"] = train_data[col]**2
    train_data[f"{col}^3"] = train_data[col]**3

feature_cols = [col for col in test_data.columns if col not in ['id','time', 'Y1', 'Y2']]
for col in feature_cols:
    test_data[f"{col}_rollmean_{window_size}"] = test_data[col].rolling(window=window_size).mean()
for col in feature_cols:
    values = test_data[col].to_numpy()
    slopes = [np.nan] * (window_size-1)
    x = np.arange(window_size)
    for i in range(window_size -1,len(values)):
        y = values[i-window_size+1:i+1]
        slope = np.polyfit(x,y,1)[0]
        slopes.append(slope)
    test_data[f"{col}_rollslope_{window_size}"] = pd.Series(slopes, index=test_data.index)
for col in feature_cols:
    test_data[f"{col}_rollvol_{window_size}"] = (
        test_data[col]
        .rolling(window=window_size, min_periods=window_size)
        .std(ddof=0)
    )
for col in feature_cols:
    for lag in lags:
        test_data[f"{col}_lag{lag}"] = test_data[col].shift(lag)
for col in feature_cols:
    test_data[f"{col}^2"] = test_data[col]**2
    test_data[f"{col}^3"] = test_data[col]**3

C:\Users\JonathanXu\AppData\Local\Temp\ipykernel_24380\4008600144.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data[f"{col}^2"] = train_data[col]**2
C:\Users\JonathanXu\AppData\Local\Temp\ipykernel_24380\4008600144.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data[f"{col}^3"] = train_data[col]**3
C:\Users\JonathanXu\AppData\Local\Temp\ipykernel_24380\4008600144.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

In [246]:
print(train_data.shape)
display(train_data)
print(test_data.shape)
display(test_data)

(80000, 129)


,time,A,B,C,D,E,F,G,H,I,...,J^2,J^3,K^2,K^3,L^2,L^3,M^2,M^3,N^2,N^3
0,0,0.207366,-0.159951,-0.634176,-0.580962,-0.266505,0.060173,-0.475257,-1.486516,-0.332594,...,0.450867,-0.302742,0.051143,-0.011566,0.035203,-0.006605,0.608769,-0.474984,0.617741,-4.855231e-01
1,1,0.188828,-0.265508,0.042143,-0.550442,-0.132319,-0.185219,0.028295,0.093210,-0.518139,...,0.063462,-0.015987,0.120996,-0.042088,0.128930,-0.046295,0.026003,-0.004193,0.000416,8.490895e-06
2,2,-0.144261,-0.577142,-0.214634,-0.747391,-0.184255,-0.464831,-0.085181,0.700449,-0.603438,...,0.039114,0.007736,0.321144,-0.181991,0.337327,-0.195919,0.041098,0.008332,0.018295,2.474656e-03
3,3,0.208982,-0.310449,0.513708,-0.562868,0.742308,-0.305487,0.762246,1.363020,-0.384575,...,0.276209,0.145164,0.121462,-0.042331,0.183269,-0.078457,0.301394,0.165463,0.221870,1.045077e-01
4,4,0.093320,-0.358156,0.173188,-0.687296,-0.161461,-0.116062,-0.245748,0.863372,-0.655588,...,0.069358,-0.018266,0.310726,-0.173207,0.231567,-0.111433,0.006989,0.000584,0.000010,2.941585e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,80000,-0.339802,-0.372094,-0.120952,-0.363438,0.266299,-0.222866,-0.000358,0.470755,-0.412086,...,0.110507,-0.036735,0.140144,-0.052464,0.063044,-0.015829,0.003460,-0.000204,0.097862,3.061388e-02
79996,80001,-0.421921,-0.222554,-0.689585,-0.512266,-0.206562,-0.494450,-0.197705,-0.566908,-0.570731,...,1.196243,-1.308365,0.146662,-0.056166,0.062290,-0.015546,0.199282,-0.088962,0.277780,-1.464038e-01
79997,80002,-0.467038,-0.282018,0.010707,-0.589594,0.067635,-0.410820,0.032695,-0.506164,-0.513340,...,0.759655,-0.662101,0.119428,-0.041273,0.161896,-0.065141,0.139117,-0.051888,0.176311,-7.403190e-02
79998,80003,-0.369710,-0.322384,-0.540326,-0.563206,-0.568736,-0.550024,-0.339077,-0.903734,-0.212600,...,0.326519,-0.186579,0.050047,-0.011196,0.094364,-0.028987,1.118363,-1.182699,4.050212,-8.151107e+00


(15996, 128)


,id,time,A,B,C,D,E,F,G,H,...,J^2,J^3,K^2,K^3,L^2,L^3,M^2,M^3,N^2,N^3
0,1,80005,-0.371888,-0.273485,0.538326,-0.377691,1.283159,-0.286957,0.439415,0.020831,...,0.013840,0.001628,0.136826,-0.050612,0.143567,-0.054398,0.221056,0.103933,0.169576,0.069831
1,2,80006,-0.459598,-0.514915,-0.235153,-0.262379,0.044343,-0.429888,-0.059034,-0.725713,...,0.065384,-0.016719,0.213353,-0.098548,0.062258,-0.015534,0.046978,-0.010182,0.107978,-0.035481
2,3,80007,-0.381609,-0.265023,-0.629131,-0.186288,-0.146302,-0.357875,0.114223,-0.359341,...,0.163283,-0.065980,0.074068,-0.020158,0.037310,-0.007207,0.461518,-0.313533,0.226891,-0.108075
3,4,80008,-0.371423,-0.106279,-1.142702,-0.200429,-0.176918,-0.354048,-0.230199,-0.305226,...,0.110895,-0.036929,0.269027,0.139539,0.000828,-0.000024,0.372857,-0.227674,0.717023,-0.607156
4,5,80009,-0.309393,-0.015144,-1.099371,-0.405841,-0.239068,-0.403306,-0.983134,-1.067315,...,0.656016,-0.531339,0.034298,0.006352,0.022576,-0.003392,1.166811,-1.260378,0.470508,-0.322738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15991,15992,95996,-0.453361,-0.196443,-0.050846,-0.447012,-0.077724,-0.592894,-0.161817,-0.511905,...,0.195880,-0.086694,0.046734,-0.010103,0.014885,0.001816,0.487012,-0.339867,0.011016,0.001156
15992,15993,95997,-0.481709,-0.258110,-0.454017,-0.603110,0.040541,-0.508827,-0.062558,-0.437095,...,0.108496,-0.035737,0.184528,-0.079267,0.119615,-0.041369,0.008538,0.000789,0.074819,-0.020465
15993,15994,95998,-0.457018,-0.300467,0.403098,-0.555525,0.255084,-0.561159,0.126289,0.315079,...,0.139156,0.051910,0.044152,-0.009277,0.017341,0.002283,0.435206,0.287106,0.016586,0.002136
15994,15995,95999,-0.366164,-0.059930,0.541362,-0.505980,0.351848,-0.450031,-0.220432,0.985248,...,0.832400,0.759449,0.193196,-0.084918,0.549195,0.406995,1.735722,2.286758,0.072308,0.019444


In [247]:
x = train_data.drop(columns=["Y1", "Y2"])
y1 = train_data["Y1"]
y2 = train_data["Y2"]

x_train, x_val, y1_train, y1_val = train_test_split(x, y1, test_size=0.25, shuffle=False)
_, _, y2_train, y2_val = train_test_split(x, y2, test_size=0.25, shuffle=False)

### Train model

In [248]:
params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": -1
}

# model for Y1
y1_train_data = lgb.Dataset(x_train, label=y1_train)
y1_val_data = lgb.Dataset(x_val, label=y1_val, reference=y1_train_data)

model_y1 = lgb.train(
    params,
    y1_train_data,
    valid_sets=[y1_train_data, y1_val_data],
    num_boost_round=2000,
    callbacks=[
        lgb.early_stopping(50),
        lgb.log_evaluation(200)
    ]
)

# model for Y2
y2_train_data = lgb.Dataset(x_train, label=y2_train)
y2_val_data = lgb.Dataset(x_val, label=y2_val, reference=y2_train_data)

model_y2 = lgb.train(
    params,
    y2_train_data,
    valid_sets=[y2_train_data, y2_val_data],
    num_boost_round=2000,
    callbacks=[
        lgb.early_stopping(50),
        lgb.log_evaluation(200)
    ]
)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[118]	training's rmse: 0.334882	valid_1's rmse: 0.630568
Training until validation scores don't improve for 50 rounds
[200]	training's rmse: 0.258031	valid_1's rmse: 0.670545
Early stopping, best iteration is:
[190]	training's rmse: 0.261226	valid_1's rmse: 0.67006


### Feature selection

In [ ]:
def get_modified_models(top_k: int = 20):

    def get_feature_importance(model):
        importance=  model.feature_importance(importance_type='gain')
        feature_importance = pd.DataFrame({
            'feature': x_train.columns,
            'importance': importance
        }).sort_values('importance', ascending=False)

        return feature_importance.head(top_k)['feature'].tolist()

    selected_feature_y1 = get_feature_importance(model_y1)
    selected_feature_y2 = get_feature_importance(model_y2)

    # build new model based on selected features
    selected_x_train_y1 = lgb.Dataset(x_train[selected_feature_y1], label=y1_train)
    selected_val_y1 = lgb.Dataset(x_val[selected_feature_y1], label=y1_val, reference=selected_x_train_y1)

    selected_x_train_y2 = lgb.Dataset(x_train[selected_feature_y2], label=y2_train)
    selected_val_y2 = lgb.Dataset(x_val[selected_feature_y2], label=y2_val, reference=selected_x_train_y2)

    modified_model_y1 = lgb.train(
        params,
        selected_x_train_y1,
        valid_sets=[selected_x_train_y1, selected_val_y1],
        num_boost_round=2000,
        callbacks=[
            lgb.early_stopping(50),
            lgb.log_evaluation(200)
        ]
    )

    modified_model_y2 = lgb.train(
        params,
        selected_x_train_y2,
        valid_sets=[selected_x_train_y2, selected_val_y2],
        num_boost_round=2000,
        callbacks=[
            lgb.early_stopping(50),
            lgb.log_evaluation(200)
        ]
    )
    return modified_model_y1, modified_model_y2, selected_feature_y1, selected_feature_y2   

### Evaluation and Prediction

In [249]:
val_pred_y1 = model_y1.predict(x_val)
val_pred_y2 = model_y2.predict(x_val)

print("R2 for Y1: ", r2_score(y1_val, val_pred_y1))
print("R2 for Y2: ", r2_score(y2_val, val_pred_y2))

R2 for Y1:  0.7115499232134459
R2 for Y2:  0.648389212922967


In [250]:
test_pred_y1 = model_y1.predict(test_data.drop(columns=["id"]))
test_pred_y2 = model_y2.predict(test_data.drop(columns=["id"]))

### Submission data

In [251]:
submission = pd.DataFrame({
    "id": test_data["id"],
    "Y1": test_pred_y1,
    "Y2": test_pred_y2
})

import datetime
import os
now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

path = os.path.join(r"submissions", f"submission{now}.csv")
submission.to_csv(path, index=False)
print("saved to csv file")

saved to csv file
